In [1]:
import pandas as pd


In [2]:
df = pd.read_parquet('../../data/processed/challenge_campus_biomedico_2024_cleaned.parquet')

In [4]:
def print_details_corrections (df, code, description, code_groups, description_groups):
    
    not_unique = False

    for cod, desc_count in code_groups.items():
        if desc_count > 1:
            associated_descriptions = df[df[code] == cod][description].unique()
            print(f"The {cod} code is associated with {desc_count} descriptions: {associated_descriptions}")
            not_unique = True
            
        
    for desc, code_count in description_groups.items():
        if code_count > 1:
            associated_codes =df[df[description] == desc][code].unique()
            print(f"The {desc} description is associated with {code_count} codes: {associated_codes}")
            not_unique = True

    if not_unique:
        print(f"--> NOT unique correlation between {code} and {description}\n")



In [5]:
def remove_columns_with_unique_correlation(df, columns_pairs) -> pd.DataFrame:
    pairs_removed = []

    for code, description in columns_pairs:
        if code in df.columns and description in df.columns:
            code_groups = df.groupby(code)[description].nunique()
            description_groups = df.groupby(description)[code].nunique()

            print_details_corrections(df, code, description, code_groups, description_groups)

            unique_correlation_code_description = all(code_groups <= 1)
            unique_correlation_description_code = all(description_groups <= 1)

            if unique_correlation_code_description and unique_correlation_description_code:
                df.drop(columns=[code], inplace=True)
                print(f'Unique correlation between {code} and {description}. Column {code} removed.')
                pairs_removed.append((code, description))
        else:
            print(f'Columns {code} or {description} not found in the dataframe.')
            pairs_removed.append((code, description))

    # Update the list of columns pairs removing the ones that have been removed
    columns_pairs_updated = [pair for pair in columns_pairs if pair not in pairs_removed]
    return df, columns_pairs_updated

            

In [7]:
print("------------ check unique correlation between codes and descriptions: ------------\n")
df, columns_pairs_updated = remove_columns_with_unique_correlation(df, columns_pairs)

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'ora_inizio_erogazione', 'ora_fine_erogazione',
       'data_disdetta'],
      dtype='object')
------------ check unique correlation between codes and descriptions: ------------

Columns codice_provincia_residenza or provincia_residenza not found in the dataframe.
Columns codice_provincia_erogazione or provincia_erogazione not found in the dataframe.
Columns codice_regione_residenza or regione_residenza not found in the dataframe.
Columns codice_asl_residenza or asl_residenza not found in the da

KeyError: "['codice_provincia_residenza'] not found in axis"

In [ ]:
def clean_codice_struttura_erogazione(df, column = 'codice_struttura_erogazione'):
    df[column] = df[column].astype(int)
    return df

In [ ]:
print(df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALERO CENTRALE TARANTO'].codice_struttura_erogazione.sort_values(ascending=True).unique())


df = clean_codice_struttura_erogazione(df)

print('after')
print(df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALERO CENTRALE TARANTO'].codice_struttura_erogazione.sort_values(ascending=True).unique())

[160172.01 160172.03 160172.04]
after
[160172]


In [ ]:
print("------------ check unique correlation between codes and descriptions with cleared codice_struttura: ------------\n")
columns_pairs_updated = remove_columns_with_unique_correlation(df, columns_pairs_updated)

------------ check unique correlation between codes and descriptions with cleared codice_struttura: ------------

The PRESIDIO OSPEDALIERO UNIFICATO description is associated with 2 codes: [ 70001 100803]
--> NOT unique correlation between codice_struttura_erogazione and struttura_erogazione



In [ ]:
columns_pairs_updated

[('codice_struttura_erogazione', 'struttura_erogazione')]

In [ ]:
df[df['struttura_erogazione'] == 'PRESIDIO OSPEDALIERO UNIFICATO']['codice_struttura_erogazione'].value_counts()

codice_struttura_erogazione
70001     1665
100803      60
Name: count, dtype: int64

In [ ]:
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'ora_inizio_erogazione', 'ora_fine_erogazione',
       'data_disdetta'],
      dtype='object')

In [ ]:
remove_data_disdetta(df)
df.columns

Index(['id_prenotazione', 'id_paziente', 'data_nascita', 'sesso',
       'regione_residenza', 'asl_residenza', 'provincia_residenza',
       'comune_residenza', 'tipologia_servizio', 'descrizione_attivita',
       'data_contatto', 'regione_erogazione', 'asl_erogazione',
       'provincia_erogazione', 'struttura_erogazione',
       'codice_struttura_erogazione', 'tipologia_struttura_erogazione',
       'id_professionista_sanitario', 'tipologia_professionista_sanitario',
       'data_erogazione', 'ora_inizio_erogazione', 'ora_fine_erogazione'],
      dtype='object')